In [1]:
from gensim.models import Word2Vec
import pickle
import numpy as np
import pandas as pd
import os
from gensim.test.utils import get_tmpfile
import os

c:\users\costanza\appdata\local\programs\python\python37-32\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from gensim.parsing.preprocessing import preprocess_string
import re
import string
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import strip_numeric
def remove_ip(s):
    # Replace all ip adresses with '<ip>' tag
    ip_regexp = r"\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b"
    return re.sub(ip_regexp, '<ip>', s)
def remove_email(s):
    # Replace all email adresses with '<email>' tag
    email_regexp = r"([a-zA-Z0-9_\-\.]+)@([a-zA-Z0-9_\-\.]+)\.([a-zA-Z]{2,5})"
    return re.sub(email_regexp, '<email>', s)
def remove_mailto(s):
    # Replace all "<mailto:<email>>" with <email>. Email adresses should be replaced by remove_email first.
    return s.replace("<mailto:<email>>", "<email>")
def remove_url(s):
    # Replace all url's with '<url>' tag
    url_regexp = r"((http|ftp|https):\/\/)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)"
    s = re.sub(url_regexp, '<url>', s)
    # Sometimes url's are inside <> so we need to replace <<url>> with <url>
    return s.replace("<<url>>", "<url>")
def remove_punc(s, exceptions):
    # Remove all punctuation from string with exceptions in list exceptions
    remove = string.punctuation
    for exception in exceptions:
        remove = remove.replace(exception, "")
    # Create the pattern
    pattern = r"[{}]".format(remove)

    return re.sub(pattern, "", s)
def remove_custom_stopwords(s, stopwords):
    for stopword in stopwords:
        s = s.replace(stopword, "")
    return s
def lower_case(s):
    return s.lower()
def preprocess_sentence_fn(s):
    # Preprocess a single sentence to a list of tokens
    punc_exceptions = ['<', '>']
    custom_stopwords = ['dear', 'sincerely', 'thanks', 'yours', 'regards']
    filters = [lower_case,
               remove_ip,
               remove_email,
               remove_mailto,
               #remove_url,
               lambda x: remove_punc(x, punc_exceptions),
               remove_stopwords,
               lambda x: remove_custom_stopwords(x, custom_stopwords),
               strip_multiple_whitespaces,
               stem_text,
               strip_numeric]
    out = preprocess_string(s, filters=filters)
    return out
def preprocess_docs_fn(docs):
    # Apply preprocess_sentence_fn to a list of sentances (docs) to get a list of lists
    return [preprocess_sentence_fn(s) for s in docs]

In [10]:
# Read in the data
ticket_dat = pd.read_csv('../data/12-04-ticket_dat.csv')
faq_dat = pd.read_csv('../data/12-04-faq_dat.csv')
# Replace the NaNs
ticket_dat.fillna('', inplace=True)
faq_dat.fillna('', inplace=True)

In [ ]:
# FAQ question
faq_ques = list(faq_dat.question)
n_faq_ques = len(faq_ques)
# FAQ answer
faq_ans = list(faq_dat.answer_title + " " + faq_dat.answer)
n_faq_ans = len(faq_ans)
#ticket question
ticket_ques = list(ticket_dat.question)
n_ticket_ques = len(ticket_ques)
#ticket ans
ticket_ans = list(ticket_dat.answer)
n_ticket_ans = len(ticket_ans)

## With manually written key words:

In [11]:
#FAQ KEYWORDS
faq_kw = list(faq_dat.key_word)
#preprocess the key words
faq_kw_prepro = preprocess_docs_fn(faq_kw)

In [22]:
# Model assumption: two different embeddings
all_ans = faq_ans + ticket_ans
all_ans_prepro = preprocess_docs_fn(all_ans)
ticket_ques_prepro = preprocess_docs_fn(ticket_ques)

In [20]:
#flatten key word list
faq_kw_list = [item for sublist in faq_kw_prepro for item in sublist]
faq_kw_list

['phish',
 'blacklist',
 'print',
 'monei',
 'print',
 'neg',
 'permiss',
 'vpn',
 'sset',
 'whitelist',
 'vpn',
 'instal',
 'instal',
 'id',
 'servic',
 'desk',
 'stuck',
 'payment',
 'new',
 'password',
 'password',
 'come',
 'password',
 'id',
 'matlab',
 'instal',
 'edoz',
 'phone',
 'number',
 'zue',
 'entranc',
 'new',
 'password',
 'offic',
 'address',
 'matricul',
 'number',
 'clear',
 'balanc',
 'restor',
 'certif',
 'download',
 'offic',
 'internation',
 'navig',
 'link',
 'backup',
 'clinet',
 'malwar',
 'clean',
 'malwar',
 'clean',
 'depart',
 'delat',
 'cq',
 'manual',
 'emb',
 'cq',
 'mail',
 'list',
 'chang',
 'password',
 'log',
 'mailbox',
 'quota',
 'extend',
 'mailbox',
 'quota',
 'employe',
 'mailbox',
 'quota',
 'student',
 'mail',
 'header',
 'mail',
 'list',
 'creation',
 'offic',
 'activaiton',
 'contact',
 'isg',
 'csr',
 'edit',
 'homepag',
 'pdf',
 'sharepoint',
 'breadcrumb',
 'firefox',
 'login',
 'hide',
 'column',
 'sharepoint',
 'confer',
 'number',
 'r

In [27]:
def check(sentence, kw):
    S1 = set(sentence)
    S2 = set(kw)
    inter = S1.intersection(S2)
    return bool(inter) #true if not empty

In [28]:
pre_class = np.empty((len(ticket_ques_prepro), 1), dtype=int)
for i in range(len(ticket_ques_prepro)):
    present = check(sentence=ticket_ques_prepro[i], kw=faq_kw_list)
    if present:
        pre_class[i] = 0
    else: 
        pre_class[i] = -1 

In [33]:
print(list(pre_class).count(-1))
print(len(pre_class))
#this method doesn't discriminate enough...

96
4206


## With most important words from FAQ: